### 1.生成数据集

In [1]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)


### 2.读取数据

In [3]:
import torch.utils.data as Data

batch_size=10
#将训练数据的特征和标签组合
dataset=Data.TensorDataset(features,labels)
#随机读取小批量
data_iter=Data.DataLoader(dataset,batch_size,shuffle=True)

In [4]:
#打印第一个小批量样本
for X,y in data_iter:
    print(X,y)
    break

tensor([[-0.7198, -1.9864],
        [ 0.5687,  1.2543],
        [ 0.2994, -0.4906],
        [-0.8062, -0.1374],
        [-0.1805,  2.5164],
        [ 2.5531,  0.0861],
        [-0.3419, -0.8217],
        [ 0.1597, -0.8209],
        [-0.7413,  1.4990],
        [ 0.3202,  2.0111]]) tensor([ 9.5106,  1.0657,  6.4527,  3.0551, -4.7301,  9.0256,  6.3105,  7.3172,
        -2.3802, -1.9985])


### 3.定义模型

In [7]:
#继承自nn.Module,既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络
from torch import nn
class LinearNet(nn.Module):
    def __init__(self,n_features):
        super(LinearNet,self).__init__()
        self.linear=nn.Linear(n_features,1)#隐藏层数为1
    #forward 定义前向传播
    def forward(self,x):
        y=self.linear(x)
        return y

net=LinearNet(num_inputs)
print(net)#打印网络结构
        

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [8]:
#还可以用nn.Sequential来更加方便地搭建网络，Sequential是一个有序的容器，网络层将按照在传入Sequential的顺序依次被添加到计算图中。
net=nn.Sequential(
    nn.Linear(num_inputs,1)
    #此处还可以传入其他层
)
print(net)
print(net[0])

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [9]:
#通过net.parameters()来查看模型所有的可学习参数，此函数将返回一个生成器。
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.1297, 0.0274]], requires_grad=True)
Parameter containing:
tensor([-0.6666], requires_grad=True)


### 4.初始化模型参数

In [ ]:
#过init.normal_将权重参数每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布。偏差会初始化为零。
from torch.nn import init

init.normal_(net[0].weight,mean=0,std=0.01)
init.constant_(net[0].bias,val=0)

#如果是用一开始的方法定义模型
#net[0].weight要改成net.linear.weight,bias同理

### 5.定义损失函数
PyTorch将损失函数实现为nn.Module的子类。我们现在使用它提供的均方误差损失作为模型的损失函数。

In [15]:
loss=nn.MSELoss()

### 6.定义优化算法
torch.optim模块提供了很多常用的优化算法比如SGD、Adam和RMSProp等。下面我们创建一个用于优化net所有参数的优化器实例，并指定学习率为0.03的小批量随机梯度下降（SGD）为优化算法。

In [10]:
import torch.optim as optim

optimizer=optim.SGD(net.parameters(),lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [13]:
#还可以为不同子网络设置不同的学习率，这在finetune时经常用到。例：
optimizer =optim.SGD([
                # 如果对某个参数不指定学习率，就使用最外层的默认学习率
                {'params': net.subnet1.parameters()}, # lr=0.03
                {'params': net.subnet2.parameters(), 'lr': 0.01}
            ], lr=0.03)


AttributeError: 'Sequential' object has no attribute 'subnet1'

In [14]:
#修改学习率
#主要有两种做法。一种是修改optimizer.param_groups中对应的学习率，
#另一种是更简单也是较为推荐的做法——新建优化器，由于optimizer十分轻量级，构建开销很小，故而可以构建新的optimizer。
#但是后者对于使用动量的优化器（如Adam），会丢失动量等状态信息，可能会造成损失函数的收敛出现震荡等情况。
for param_group in optimizer.param_groups:
    param_group['lr'] *=0.1 #学习率为之前的0.1倍

### 7.训练模型
通过调用optim实例的step函数来迭代模型参数。按照小批量随机梯度下降的定义，我们在step函数中指明批量大小，从而对批量中样本梯度求平均。

In [17]:
num_epochs=3
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        output=net(X)
        l=loss(output,y.view(-1,1))
        optimizer.zero_grad()#梯度清零
        l.backward()
        optimizer.step()
    print('epoch %d,loss:%f' %(epoch,l.item()))

epoch 1,loss:2.489166
epoch 2,loss:1.055176
epoch 3,loss:0.297912


In [18]:
dense=net[0]
print(true_w,dense.weight)
print(true_b,dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.8581, -3.0703]], requires_grad=True)
4.2 Parameter containing:
tensor([3.7928], requires_grad=True)
